In [7]:
library('r2excel')
library('magrittr')
library('data.table')

In [60]:
data = fread('./output-data/big-mac-full-index.csv') %>%
    .[, .(
        Country = name,
        iso_a3,
        currency_code,
        local_price,
        dollar_ex,
        dollar_price,
        dollar_ppp = dollar_ex * dollar_price / .SD[currency_code == 'USD']$dollar_price,
        dollar_valuation = USD_raw,
        euro_valuation = EUR_raw,
        sterling_valuation = GBP_raw,
        yen_valuation = JPY_raw,
        yuan_valuation = CNY_raw,
        dollar_adj_valuation = USD_adjusted,
        euro_adj_valuation = EUR_adjusted,
        sterling_adj_valuation = GBP_adjusted,
        yen_adj_valuation = JPY_adjusted,
        yuan_adj_valuation = CNY_adjusted
    ), by=date]
tail(data)

date,Country,iso_a3,currency_code,local_price,dollar_ex,dollar_price,dollar_ppp,dollar_valuation,euro_valuation,sterling_valuation,yen_valuation,yuan_valuation,dollar_adj_valuation,euro_adj_valuation,sterling_adj_valuation,yen_adj_valuation,yuan_adj_valuation
2018-01-01,Taiwan,TWN,TWD,69.00,29.55300,2.334788,13.068182,-0.558,-0.517,-0.471,-0.320,-0.264,-0.380,-0.447,-0.365,-0.192,-0.385
2018-01-01,Ukraine,UKR,UAH,47.00,28.71500,1.636775,8.901515,-0.690,-0.662,-0.629,-0.523,-0.484,NA,NA,NA,NA,NA
2018-01-01,Uruguay,URY,UYU,140.00,28.58000,4.898530,26.515152,-0.072,0.013,0.110,0.427,0.544,NA,NA,NA,NA,NA
2018-01-01,United States,USA,USD,5.28,1.00000,5.280000,1.000000,0.000,0.092,0.196,0.538,0.665,0.000,-0.108,0.025,0.304,-0.007
2018-01-01,Vietnam,VNM,VND,65000.00,22711.50000,2.861986,12310.606061,-0.458,-0.408,-0.351,-0.166,-0.098,NA,NA,NA,NA,NA
2018-01-01,South Africa,ZAF,ZAR,30.00,12.25815,2.447351,5.681818,-0.536,-0.494,-0.445,-0.287,-0.228,-0.192,-0.279,-0.171,0.054,-0.197


In [9]:
dates = data$date %>% unique
dates

[1] "2000-04-01" "2001-04-01" "2002-04-01" "2003-04-01" "2004-05-01"
 [6] "2005-06-01" "2006-01-01" "2006-05-01" "2007-01-01" "2007-06-01"
[11] "2008-06-01" "2009-07-01" "2010-01-01" "2010-07-01" "2011-07-01"
[16] "2012-01-01" "2012-07-01" "2013-01-01" "2013-07-01" "2014-01-01"
[21] "2014-07-01" "2015-01-01" "2015-07-01" "2016-01-01" "2016-07-01"
[26] "2017-01-01" "2017-07-01" "2018-01-01"

In [61]:
?xlsx.addTable

In [66]:
wb = createWorkbook(type='xls')

for(sheetDate in sort(dates, decreasing = T)) {
    dateStr = sheetDate %>% strftime(format='%b%Y')
    sheet = createSheet(wb, sheetName = dateStr)
    xlsx.addTable(wb, sheet, data[date == sheetDate, -1], row.names=FALSE, startCol=1)
}

saveWorkbook(wb, 'exceltest.xls')